In [9]:
import nltk
import random
import string
import warnings
warnings.filterwarnings('ignore')

import tkinter as tk
from tkinter import messagebox

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  


class ChatbotGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("Adeena Chatbot")
        self.master.geometry("1100x500")

        # Chat window
        self.chat_window = tk.Text(self.master, width=100, height=25)
        self.chat_window.pack(padx=10, pady=10)

        # User input box
        self.user_input = tk.Entry(self.master, width=50)
        self.user_input.pack(padx=10, pady=10)

        # Send button
        self.send_button = tk.Button(self.master, text="Send", command=self.send_message)
        self.send_button.pack(padx=10, pady=10)

        # Initialize NLP and load data
        self.init_nlp()
        self.load_data()

        # Initial greeting
        self.chat_window.insert(tk.END, "Hello, there my name is Adeena. I will answer your queries. If you want to exit, type Bye!\n")

    def load_data(self):
        # Load and preprocess data
        self.f = open('Information1.txt', 'r', errors='ignore')
        self.raw = self.f.read()
        self.raw = self.raw.lower()

        # Tokenize data
        self.sent_tokens = nltk.sent_tokenize(self.raw)
        self.word_tokens = nltk.word_tokenize(self.raw)

    def init_nlp(self):
        # Initialize Lemmatizer and punctuation remover
        self.lemmer = nltk.stem.WordNetLemmatizer()
        self.remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

        # Greeting inputs and responses
        self.GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
        self.GREETING_RESPONSES = ["hi", "hey", "nods", "hi there", "hello", "I am glad! you are talking to me"]

    def LemTokens(self, tokens):
        return [self.lemmer.lemmatize(token) for token in tokens]

    def LemNormalize(self, text):
        return self.LemTokens(nltk.word_tokenize(text.lower().translate(self.remove_punct_dict)))

    def greeting(self, sentence):
        for word in sentence.split():
            if word.lower() in self.GREETING_INPUTS:
                return random.choice(self.GREETING_RESPONSES)

    def response(self, user_response):
        chatbot_response = ''
        self.sent_tokens.append(user_response)
        TfidfVec = TfidfVectorizer(tokenizer=self.LemNormalize, stop_words="english")
        tfidf = TfidfVec.fit_transform(self.sent_tokens)
        vals = cosine_similarity(tfidf[-1], tfidf)
        idx = vals.argsort()[0][-2]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[-2]
        if req_tfidf == 0:
            chatbot_response = "I am sorry! I don't understand you."
        else:
            chatbot_response = self.sent_tokens[idx]
        self.sent_tokens.remove(user_response)  # Remove user input from the token list to avoid repetition
        return chatbot_response

    def send_message(self):
        user_response = self.user_input.get()
        self.user_input.delete(0, tk.END)
        self.chat_window.insert(tk.END, "You: " + user_response + "\n")

        if user_response.lower() != 'bye':
            if user_response.lower() == 'thanks' or user_response.lower() == 'thank you':
                self.chat_window.insert(tk.END, "Adeena: You're welcome!\n")
            else:
                if self.greeting(user_response) is not None:
                    self.chat_window.insert(tk.END, "Adeena: " + self.greeting(user_response) + "\n")
                else:
                    self.chat_window.insert(tk.END, "Adeena: " + self.response(user_response) + "\n")
        else:
            self.chat_window.insert(tk.END, "Adeena: Bye! Have a great time!\n")
            self.chat_window.update_idletasks()
            self.master.after(1500, self.master.destroy)


root = tk.Tk()
my_gui = ChatbotGUI(root)
root.mainloop()
